In [11]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
# import pandas_ta
import warnings
warnings.filterwarnings('ignore')


sp500 = pd.read_html ('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
symbols_list = sp500['Symbol'].unique().tolist()
end_date = '2023-11-19'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)
df = yf.download(tickers=symbols_list, start=start_date, end=end_date)
df

[*********************100%%**********************]  503 of 503 completed


Adj Close                                                 \
                     A        AAL        AAPL        ABBV        ABNB   
Date                                                                    
2015-11-23   37.476856  40.423805   26.799690   43.283794         NaN   
2015-11-24   38.076035  39.401276   27.056881   43.269642         NaN   
2015-11-25   38.703293  39.477722   26.863424   42.675064         NaN   
2015-11-27   39.293118  39.840862   26.813345   42.469788         NaN   
2015-11-30   39.152687  39.429932   26.924871   41.160301         NaN   
...                ...        ...         ...         ...         ...   
2023-11-13  107.459999  11.780000  184.800003  138.639999  119.150002   
2023-11-14  111.610001  12.250000  187.440002  138.059998  126.680000   
2023-11-15  113.599998  12.420000  188.009995  137.600006  128.350006   
2023-11-16  114.190002  12.190000  189.710007  138.279999  126.279999   
2023-11-17  113.150002  12.290000  189.690002  138.300003  127.150002   

                                                                       ...  \
                   ABT       ACGL         ACN        ADBE         ADI  ...   
Date                                                                   ...   
2015-11-23   39.117043  24.643333   94.891479   91.959999   48.095818  ...   
2015-11-24   39.125648  24.533333   94.460297   92.000000   51.161083  ...   
2015-11-25   39.091213  24.216667   94.240334   91.769997   51.017117  ...   
2015-11-27   39.099819  24.306667   94.557121   92.169998   51.186481  ...   
2015-11-30   38.669392  24.156668   94.345932   91.459999   52.185658  ...   
...                ...        ...         ...         ...         ...  ...   
2023-11-13   95.790001  86.570000  315.630005  590.340027  170.660004  ...   
2023-11-14   97.080002  85.500000  320.489990  604.330017  177.580002  ...   
2023-11-15   98.000000  82.660004  325.500000  595.309998  180.779999  ...   
2023-11-16  100.260002  83.709999  327.320007  602.059998  179.839996  ...   
2023-11-17   99.550003  83.599998  327.829987  602.659973  183.050003  ...   

             Volume                                                         \
               WYNN      XEL       XOM     XRAY      XYL      YUM      ZBH   
Date                                                                         
2015-11-23  2427400  2341500  11868400   616000   870600  3554005  1031648   
2015-11-24  5628300  4416300  15055500   917900  1044000  3045455  1313868   
2015-11-25  2358500  2200100   8980400   365000   916200  3222808  1008576   
2015-11-27  2285100  1468000   4156600   293200   514200  2558884   917730   
2015-11-30  2878100  3597100  17729200   981300  1470200  4477490  1600517   
...             ...      ...       ...      ...      ...      ...      ...   
2023-11-13  3779800  2960500  15308400  3955600  1261100  1230500  1702200   
2023-11-14  4061300  4250000  18260500  4303800  1500400  1353700  1909700   
2023-11-15  2544600  3439500  20137600  4925400  1297000  1309400  3648600   
2023-11-16  3022800  3970800  22469100  3552800  1314800  1554600  2300700   
2023-11-17  1678400  4327900  22984900  2413300  1286600  1089600  2991400   

                                      
              ZBRA     ZION      ZTS  
Date                                  
2015-11-23  320300  1438700  3265300  
2015-11-24  431800  1608600  1713500  
2015-11-25  292100  1086200  2196800  
2015-11-27  175700   763200  1155100  
2015-11-30  436500  2614000  3481900  
...            ...      ...      ...  
2023-11-13  411200  1240300  1544900  
2023-11-14  542700  3270200  1421500  
2023-11-15  501600  2444400  1714100  
2023-11-16  275700  2217300  1566900  
2023-11-17  340600  2136200  2425100  

[2011 rows x 3018 columns]